# Routing y Orquestacion simple (sin multi-agent complejo)

## Objetivo
Mostrar un patron pragmatico: un router selecciona el dominio correcto y delega en un Agentic RAG por dominio.

Esto evita sobredisenar un sistema multi-agente cuando el problema real es enrutamiento semantico + retrieval robusto.

In [ ]:
from pathlib import Path
import sys
import pandas as pd

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.vector_store_lab import build_index_from_json
from scripts.rag_pipelines import AgenticRAG, HeroRouterOrchestrator

OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
batman_db, _, batman_index_stats, _ = build_index_from_json(
    json_path=ROOT / 'data' / 'batman_comics.json',
    persist_dir=OUTPUTS_DIR / 'chroma_router_batman',
    collection_name='router_batman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

spiderman_db, _, spiderman_index_stats, _ = build_index_from_json(
    json_path=ROOT / 'data' / 'spiderman_comics.json',
    persist_dir=OUTPUTS_DIR / 'chroma_router_spiderman',
    collection_name='router_spiderman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print('Batman index:', batman_index_stats)
print('Spider-Man index:', spiderman_index_stats)

In [ ]:
batman_pipeline = AgenticRAG(
    vector_db=batman_db,
    model='gpt-5-mini',
    embedding_model='text-embedding-3-small',
    k=6,
)

spiderman_pipeline = AgenticRAG(
    vector_db=spiderman_db,
    model='gpt-5-mini',
    embedding_model='text-embedding-3-small',
    k=6,
)

router = HeroRouterOrchestrator(
    hero_pipelines={
        'batman': batman_pipeline,
        'spiderman': spiderman_pipeline,
    },
    default_hero='batman',
)

print('Router ready.')

## Consultas de prueba

Evaluamos mezcla de consultas directas y ambiguas para observar comportamiento de routing.

In [ ]:
queries = [
    'Como se transforma Bruce Wayne en Batman durante Year One?',
    'Explica la filosofia de responsabilidad de Peter Parker.',
    'Compara a Joker con Green Goblin en terminos de dano psicologico.',
    'Que rol estrategico cumple Batman en la Liga de la Justicia?',
    'Que aprendizaje deja la Saga del Clon para identidad y autenticidad?',
]

rows = []
for query in queries:
    result = router.run(query)
    rows.append(result)

routing_df = pd.DataFrame(rows)
routing_df

In [ ]:
summary = (
    routing_df.groupby('selected_hero', as_index=False)
    .agg(
        queries=('query', 'count'),
        avg_latency=('latency_seconds', 'mean'),
        avg_groundedness=('groundedness', 'mean'),
    )
    .round(4)
)
summary

In [ ]:
csv_path = OUTPUTS_DIR / 'routing_orchestration_results.csv'
routing_df.to_csv(csv_path, index=False)
print(f'Saved: {csv_path}')
routing_df[['query', 'selected_hero', 'route', 'pipeline', 'groundedness']]

## Conclusiones de orquestacion

- Routing + Agentic RAG por dominio es un baseline fuerte antes de escalar a sistemas multi-agente completos.
- Mantener retrievers especializados reduce ruido y mejora trazabilidad.
- La evaluacion de rutas es tan importante como la evaluacion de respuestas.